In [ ]:
%pylab inline

In [ ]:
np.set_printoptions(linewidth=120)
plt.rcParams["figure.figsize"] = (18,10)
plt.rcParams["legend.loc"] = "lower right"

import os
base_dir = '/local/scratch/toolkit/'
os.environ['EXOT_ACCESS_DIR'] = str(base_dir + "benchmark_platforms_access/")
os.chdir(base_dir + 'datapro')
print("Working directory: " + str(os.getcwd()))
print("Key directory set to: " + str(os.environ['EXOT_ACCESS_DIR']))

import toml    as TOML              # TOML library to handle config files
import pandas  as pd                # Pandas for data manipulation
import seaborn as sb                # Statistical data visualisation
import pathlib as PATH              # Used to convert strings to path objects for (de-)serialisation
import types   as tp                # Library used for online method development

# scikit-learn packets
import sklearn.base                 #
import sklearn.svm                  # LinearSVC, SVC
import sklearn.pipeline             # make_pipeline
import sklearn.preprocessing        # StandardScaler
import sklearn.decomposition        # PCA
import sklearn.naive_bayes          # GaussianNB
import sklearn.metrics              # CM

# Toolkit packets
import exot                       # Dataprocessing
import exot.experiment            # Experiment for execution
import exot.util                  # General utilities
import exot.layer                 # Data manipulation layers
from exot.util       import *
from exot.layer      import *
from exot.channel    import *
from exot.experiment import *

# Plotting
from exot.plotting.experiment_plotter import *
from exot.plotting.run_plotter        import *

In [ ]:
TEST   = False
UPDATE = True
RESUME = True

In [ ]:
environments_to_execute = [
#    'ARMv7',
    'ARMv8',
#    'Haswell',
#    'Sandy',
]

phases_cache = {
    'ARMv8':{'test':['trainShortARMv8','evalShortARMv8'],
               'normal':['trainARMv8','evalARMv8']},
    'Haswell':{'test':['trainShortHaswell','evalShortHaswell'],
               'normal':['trainHaswell','evalHaswell']}
}

phases_power = {
    'Sandy':{'normal':['train','eval'], 'test':['trainTest','evalTest']},
    'Haswell':{'normal':['train','eval'], 'test':['trainTest','evalTest']},
}

phases_capacity= {
    'ARMv7':{'normal':['sweepARMv7'], 'test':['test']},
    'ARMv8':{'normal':['sweepARMv8'], 'test':['test']},
    'Haswell':{'normal':['sweepHaswell'], 'test':['test']},
    'Sandy':{'normal':['sweepSandy'], 'test':['test']},
}

phases_thermal = {
    'Sandy':{'normal':['train','eval'], 'test':['trainTest','evalTest']},
    'Haswell':{'normal':['train','eval'], 'test':['trainTest','evalTest']},
    'ARMv8':{'normal':['train','eval'], 'test':['trainTest','evalTest']},
    'ARMv7':{'normal':['train','eval'], 'test':['trainTest','evalTest']},
}

In [ ]:
raise Exception("Are you sure?!")

### Cache capacity

In [ ]:
experiment = ExploratoryExperiment.read(Path("./data/cache-cc_capacity/_experiment.pickle"))

experiment.config["EXPERIMENT"]["PHASES"]["run"] = {}
experiment.config["EXPERIMENT"]["PHASES"]["run"]["repetitions"] = 1
experiment.config["EXPERIMENT"]["PHASES"]["run"]["rdpstreams"] = [
    pandas.DataFrame({"timestamp": np.arange(0, 5), "value": np.arange(0, 5)})
]
experiment.execute_in_environment(env, ['run'], resume=UPDATE and RESUME)

### Cache performance

In [ ]:
for experiment_cache_path in [
    "data/flush+flush-cc_performance/_experiment.pickle",
    "data/flush+reload-cc_performance/_experiment.pickle",
    "data/flush+prefetch-cc_performance/_experiment.pickle",
    "data/flush+flush-cc_interference/_experiment.pickle",
    "data/flush+reload-cc_interference/_experiment.pickle",
    "data/flush+prefetch-cc_interference/_experiment.pickle"
]:
    try:
        path_to_serialised_experiment = PATH.Path(experiment_cache_path)
        experiment_cache = PerformanceExperiment.read(path_to_serialised_experiment, diff_and_replace=True)
        try:
            experiment_cache.logger.parent.handlers[0].setLevel(20)
        except:
            pass
        if UPDATE:
            experiment_cache.config.experiment_exists_action = "update"
        experiment_cache.infer_execution_status()
        for env in environments_to_execute:
            experiment_cache.execute_in_environment(
                env,
                phases_cache[env]['test' if TEST else 'normal'],
                resume=UPDATE and RESUME
            )
        experiment_cache.write()
    except Exception as e:
        print("ERROR - ", experiment_cache_path, str(e))
        !ssh -i /local/scratch/toolkit/benchmark_platforms_access/id_ed25519 exot@Gimli "sudo reboot"
        !sleep 3m

### Thermal capacity

In [ ]:
for experiment_thermal_path in [
    "data/thermal-cc_capacity/_experiment.pickle",
]:
    try:
        path_to_serialised_experiment = PATH.Path(experiment_thermal_path)
        experiment_thermal = FrequencySweepExperiment.read(path_to_serialised_experiment, diff_and_replace=True)
        try:
            experiment_thermal.logger.parent.handlers[0].setLevel(20)
        except:
            pass
        if UPDATE:
            experiment_thermal.config.experiment_exists_action = "update"
        experiment_thermal.infer_execution_status()
        for env in environments_to_execute:
            experiment_thermal.execute_in_environment(
                env,
                phases_capacity[env]['test' if TEST else 'normal'],
                resume=UPDATE and RESUME
            )
        experiment_thermal.write()
    except Exception as e:
        print("ERROR - ", experiment_thermal_path, str(e))

### Thermal performance

In [ ]:
for experiment_thermal_path in [ 
    "data/thermal-cc_performance_masti/_experiment.pickle",
    "data/thermal-cc_interference/_experiment.pickle",
    "data/thermal-cc_performance/_experiment.pickle",
]:
    try:
        path_to_serialised_experiment = PATH.Path(experiment_thermal_path)
        experiment_thermal = PerformanceExperiment.read(path_to_serialised_experiment, diff_and_replace=False)
        try:
            experiment_thermal.logger.parent.handlers[0].setLevel(20)
        except:
            pass
        if UPDATE:
            experiment_thermal.config.experiment_exists_action = "update"
        experiment_thermal.infer_execution_status()
        for env in environments_to_execute:
            experiment_thermal.execute_in_environment(
                env,
                phases_thermal[env]['test' if TEST else 'normal'],
                resume=UPDATE and RESUME
            )
        experiment_thermal.write()
    except Exception as e:
        print("ERROR - ", experiment_thermal_path, str(e))